# Constant selection
![constant selection](../resources/constant_selection.png)

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import ipywidgets as widgets
%matplotlib inline
%matplotlib widget

## The Moran model + constant selection


The difference, the _only_ difference, between this and the original Moran model is that we now consider fitness in who we select when drawing an individual to reproduce. Before we considered each individual equally but now we weight individuals of each species using their fitness. 

we represent this weighting by the `fit_ratio` parameter which described the ratio between species 1 fitness($k_1$) and species 2 fitness ($k_2$). Inorder to do this weighting without violating the rule that the probabilities of each species must add up to 1, we do the weighting like this:
$$
P_{birth}(S=1) = \frac{(k_1/k_2) f_1}{(k_1/k_2) f_1 + f_2}
$$

This is the only change we have to make to the basic Moran model to introduce constant selection, and we will never have to consider the individual species' fitnesses on their own.

## Temporally varying selection

Vellend also introduces a 'temporally varying' selection. He introduces this after frequency dependence but his model is more like this constant selection model so we will consider that here. In Vellend's description, the change in time comes as a change in the fitness ratio that depends on the simulated year.

First let's say that we've arrived at our fitness ratio, we'll call this $\gamma$
\begin{align}
\frac{k_1}{k_2} = \gamma
\end{align}

We do the temporal variation, at least the simple example vellend shows, by flipping the ratio every 10 years. In general we flip the ratio every $y_{flip}$ years. When we've gone through $y_{flip}$ years we simple redefine the fitness ratio as:
\begin{align}
\frac{k_1}{k_2} = \gamma^{-1}
\end{align}

I've represented this in the code with a variable called `time_dep` that can either be `True` or `False`. If its true, we do the 10 year flip as shown in the chapter.

In [6]:
def moran_constant_selection(n_indiv, n_years, init_f1, fit_ratio, time_dep):
    """ Implements the Moran model including constant selection and simulates it in time. 
        
        Parameters
        ----------
        n_indiv: int
            number of individuals in the community
        n_years: int
            number of potential 'turnovers' of the community
        init_f1: float
            initial frequency of species 1
        fit_ratio: float
            ratio of fitness of species 1 to fitness of species 2
            
        Return
        ------
        moran: np.array (n_years, 2)
            contains the species frequencies for each year of the simulation """

    # set up an empty array for the simulated frequencies
    # initialize with the given frequency. python counts from 0, sorrryyyy
    moran = np.zeros((n_years, 2))
    moran[0] = np.array([init_f1, 1 - init_f1])

    # get a vector representing the community. vellend calls this COM
    # this just makes a random vector drawn from the initial frequency
    comm = np.random.choice([0, 1], size=n_indiv, replace=True, p=moran[0])

    # now we can do the loop as in vellend. I'm going to write it a bit differently
    # but the idea is basically the same. (yes its slower)
    for year in range(1, n_years):

        # if time dependence is true we flip the fitness ratio every ten years
        if time_dep and year % 10 == 0:
            fit_ratio = fit_ratio**-1

        # for each year we potentially replace each individual so we can loop
        # over that as well
        for indiv in range(n_indiv):
            # get probability of species 1
            f1 = np.sum(comm == 0) / n_indiv
            pr1 = fit_ratio * f1 / (fit_ratio * f1 + (1 - f1))

            # replace one individual with another
            death = np.random.randint(n_indiv)
            birth = np.random.choice([0, 1], p=[pr1, 1 - pr1])
            comm[death] = birth
        
        # when we're done looping over all of the individuals we can update
        # frequencies for the year (the timescale we care about)
        f1 = np.sum(comm == 0) / n_indiv
        moran[year] = np.array([f1, 1 - f1])

    return moran

## Running the simulations and visualization
As before we have a bunch of messy code to draw the plots. Once again we can change parameter values in the first block and not worry about the rest (unless you want to!)

In [8]:
def draw_simulation(iterations=10, individuals=250, years=50, initial_freq=0.5, fit_ratio=1, time_dep=False):
    # the plot bit, this just makes a blank plot
    fig, ax = plt.subplots(ncols=2, figsize=(10,4), sharey=True, gridspec_kw = {'width_ratios':[3, 1]})
    # trajectory labels
    ax[0].set_xlabel('Years')
    ax[0].set_ylabel('Species 1 frequency')
    ax[0].set_ylim([0, 1])
    # distribution labels and bins
    hist_bins = np.arange(0, 1.1, 0.1)
    ax[1].set_xlabel('Count')
    ax[1].set_xlim([0, iterations])

    # we're going to plot a distribution too need an array for it
    dist = np.zeros(iterations)

    # we run the simulation and draw a trajectory
    for i in tqdm(range(iterations)):
        simulation = moran_constant_selection(individuals, years, initial_freq, fit_ratio, time_dep)
        ax[0].plot(range(years), simulation[:, 0], c='C0')
        # add final freq to our dist array
        dist[i] = simulation[-1, 0]

    # plot the distribution too
    ax[1].hist(dist, bins=hist_bins, orientation='horizontal')
    ax[1].axhline(np.mean(dist), linestyle='--')


    plt.tight_layout()    
    plt.show()

# set up the interface
widgets.interact_manual(draw_simulation, iterations=(5, 50, 5), individuals=(10, 1000, 10), years=(5, 200, 5),
                  initial_freq=(0.0, 1.0, 0.01), fit_ratio=(0.5, 1.5, 0.01), time_dep=(False))

interactive(children=(IntSlider(value=10, description=&#39;iterations&#39;, max=50, min=5, step=5), IntSlider(value=25…

&lt;function __main__.draw_simulation(iterations=10, individuals=250, years=50, initial_freq=0.5, fit_ratio=1, time_dep=False)&gt;